In [1]:
from os.path import join
import pandas as pd
from prepare_data import get_station_paths, prepare_old_station_data, prepare_new_station_data

old_basepath = '../basedata/PCD Data/Data before 2020-9'
new_basepath = '../basedata/PCD Data/Data after 2020-7/PCD data after 2020-7.csv'
others_basepath = './prepared_data/others/'


In [2]:
new_columns = ['stationID', 'PM25', 'PM10', 'NO2', 'SO2', 'CO', 'O3', 'datetime_aq']
all_new_df = pd.read_csv(new_basepath, usecols=new_columns)
old_station_paths = get_station_paths(old_basepath)

# get all uncollect columns name

## new data

In [3]:
new_uncollect = set()
stations = []
columns = []
counts = []
for station in all_new_df['stationID'].unique():
    st_df = all_new_df[all_new_df.stationID == station]
    st_cols = []
    for col in st_df.columns:
        na_count = st_df[col].isna().sum()
        if na_count > 0:
            new_uncollect.add(col)
            stations.append(station)
            columns.append(col)
            counts.append(na_count)
new_uncollect_df = pd.DataFrame({'station':stations, 'column':columns, 'count':counts})
new_uncollect

{'CO', 'NO2', 'O3', 'PM10', 'PM25', 'SO2'}

In [4]:
pd.DataFrame(new_uncollect).to_csv(join(others_basepath, 'new_uncollect_columns.csv'))
new_uncollect_df.to_csv(join(others_basepath, 'new_uncollect_mapper.csv'))

## old data

### Investigate columns name

In [5]:
old_columns = set()
start_dt = set()
end_dt = set()
for station in old_station_paths:
    try:
        df = prepare_old_station_data(station)
        for col in df.columns:
            old_columns.add(col)
        start_dt.add(df.index[0])
        end_dt.add(df.index[-1])
    except:
        print(station)

33t
13t


# This start date and end date make we notice there are 1900s data

In [13]:
start_dt

{Timestamp('2001-01-10 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2010-01-01 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2010-01-02 01:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2010-01-25 02:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2010-01-26 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2010-02-08 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2010-05-05 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2010-05-07 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2011-06-28 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2011-07-01 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2011-07-13 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2011-11-08 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2013-01-19 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2014-08-22 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2015-04-09 00:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp

In [14]:
end_dt

{Timestamp('2011-06-03 14:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2011-06-26 11:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2012-10-26 10:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2012-12-31 23:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2013-07-11 09:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2013-08-31 09:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2013-09-27 07:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2014-06-25 09:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2014-07-15 23:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2014-11-23 23:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2014-12-31 23:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2015-08-26 09:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2015-10-16 16:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2015-11-30 09:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp('2015-11-30 23:00:00+0700', tz='UTC+07:00', freq='H'),
 Timestamp

In [6]:
old_columns

{' CO',
 ' CO ',
 ' Glob rad',
 ' NO ',
 ' NO2',
 ' NOX ',
 ' PM10',
 ' Pressure',
 ' RH',
 ' Rain',
 ' Rel hum',
 ' SO2 ',
 ' TEMP',
 ' Temp',
 ' Total HC',
 ' WD',
 ' WS ',
 ' Wind dir',
 ' Wind speed',
 'CH4 (ppm)',
 'CO',
 'CO(ppm)',
 'NO',
 'NO2',
 'Nox',
 'O3',
 'PM10',
 'PM10(มคก./ลบ.ม.)',
 'PM2.5',
 'PM2.5 ',
 'RH',
 'SO2',
 'TEMP',
 'TMP',
 'Temp',
 'WD',
 'WS'}

## We rename column to be the same format as this

In [7]:
col_mapper = {'CO': 'CO', ' NO2': 'NO2', ' SO2 ': 'SO2', 'O3': 'O3', ' PM10': 'PM10', ' Wind speed': 'WS', ' Wind dir': 'WD',
 ' Temp': 'Temp', ' Rel hum': 'Rain', ' PM2.5': 'PM25', 'PM10': 'PM10', 'PM2.5': 'PM25', 'NO2': 'NO2', 'SO2': 'SO2', 
 'WS': 'WS', 'WD': 'WD', 'TEMP': 'Temp', 'RH': 'Rain', 'PM2.5 ': 'PM25', ' CO': 'CO', ' WD': 'WD', ' WS ': 'WS', 'Temp': 'Temp',
  ' TEMP': 'Temp', ' RH': 'Rain', ' CO ': 'CO', ' Rain': 'Rain', 'CO(ppm)': 'CO', 'PM10(มคก./ลบ.ม.)': 'PM10', 'TMP': 'Temp'}
to_remove = ['NO', 'Nox', ' NO ', ' NOX ', ' Glob rad', ' Total HC', 'CH4 (ppm)']
to_have = ['CO', 'NO2', 'SO2', 'O3', 'PM10', 'WS', 'WD', 'Temp', 'Rain', 'PM25']

In [8]:
old_uncollect = set()
old_columns = {}
stations = []
columns = []
counts = []
maximum_st = len(old_station_paths)
i = 0
for station in old_station_paths:
    try:
        df = prepare_old_station_data(station)
        for col in df.columns:
            old_columns[col] = station
            na_count = df[col].isna().sum()
            if na_count > 0:
                old_uncollect.add(col)
                stations.append(station)
                columns.append(col)
                counts.append(na_count)
        # ==================================================================
        # We add this for find which columns are uncollected
        for col in list(set(to_have) - set(df.columns)):
            stations.append(station)
            columns.append(col)
        # ==================================================================
    except:
        print(station)
    i += 1
old_uncollect_df = pd.DataFrame({'station':stations, 'column':columns})
old_uncollect 

set()

In [ ]:
old_uncollect_df

,station,column
0,41t,Rain
1,41t,NO2
2,41t,WD
3,41t,Temp
4,41t,SO2
...,...,...
482,19t,WD
483,19t,Temp
484,19t,SO2
485,19t,PM25


In [ ]:
pd.DataFrame(old_uncollect).to_csv(join(others_basepath, 'old_uncollect_columns.csv'))
old_uncollect_df.to_csv(join(others_basepath, 'old_uncollect_mapper.csv'))